In [35]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import re
import chardet

In [36]:
# データベース接続
conn = sqlite3.connect('jalan.db')
cur = conn.cursor()

# テーブル作成
cur.execute('''
  CREATE TABLE IF NOT EXISTS inns (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    rating REAL NOT NULL,
    comment_count INTEGER NOT NULL
  )
''')

In [37]:
# スクレイピング対象URL
url = "https://www.jalan.net/110000/LRG_110500/?stayYear=2025&stayMonth=1&stayDay=22&stayCount=1&roomCount=1&adultNum=2&ypFlg=1&kenCd=110000&screenId=UWW1380&roomCrack=200000&lrgCd=110500&distCd=01&rootCd=04"

# データ取得
response = requests.get(url)
# エンコーディングを自動検出
detected_encoding = chardet.detect(response.content)['encoding']

# BeautifulSoupで解析、エンコーディングを指定
soup = BeautifulSoup(response.content, "html.parser", from_encoding=detected_encoding)

# 旅館情報を含む要素を取得
inns = soup.find_all("div", class_="p-yadoCassette__summary")

for inn in inns:
    try:
        name = inn.find("h2", class_="p-searchResultItem__facilityName").text.strip()
        rating_element = inn.find("span", class_="p-searchResultItem__summaryaverage-num")
        rating = float(rating_element.text) if rating_element else None  # 評価がない場合はNoneを代入

        # 口コミ件数の取得
        comment_count_element = inn.find("span", class_="p-searchResultItem__summarykuchikomi__totalNumber")
        if comment_count_element:
            comment_count_text = comment_count_element.text
            comment_count_match = re.search(r'\d+', comment_count_text)
            comment_count = int(comment_count_match.group(0)) if comment_count_match else None  # 口コミ件数が取得できない場合はNoneを代入
        else:
            comment_count = None

    except AttributeError:
        print("Error: Could not find element. Skipping...")
        # エラーが発生した旅館の情報をprintするなどして、原因を調査する
        print(inn)

In [38]:
# コミット
conn.commit()

# データベース接続を閉じる
conn.close()